**REPOSITORIO UNSAAC**

- Alex Jesus Flores Taco   (20210833)



In [1]:
# Cargando librerías o módulos pertinentes
import requests
from bs4 import BeautifulSoup
import re
import itertools
import pandas as pd

In [2]:
# Creando una función que permite extraer todas las urls de la tabla de datos de las
# publicaciones de una determina página
def links_scrapy(url):
    # Descargar el contenido de la página
    page = requests.get(url)
    # Crear un objeto BeautifulSoup a partir del contenido de la página
    soup = BeautifulSoup(page.content, "html.parser")
    # Encontrar el tag padre de las publicaciones que contiene su respectivo url
    publication = soup.find_all("h4", {"class":"artifact-title"})
    # Extraer los url de los tags hijos respectivos
    publication_soup = BeautifulSoup(str(publication), 'html.parser')
    a_tags = publication_soup.find_all('a') # find all anchor elements
    hrefs = ["https://repositorio.unsaac.edu.pe" + a_tag['href'] + "?show=full" for a_tag in a_tags]
    return hrefs

In [3]:
links_scrapy('https://repositorio.unsaac.edu.pe/browse?type=dateissued')

['https://repositorio.unsaac.edu.pe/handle/20.500.12918/5935?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5427?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/5796?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/6487?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/7055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/4106?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1119?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/926?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1055?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1026?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1053?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1070?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12918/1080?show=full',
 'https://repositorio.unsaac.edu.pe/handle/20.500.12